# get the sleep time for all the sleep labels after deidentification
Output a csv that contains the start and end times for a recording session

In [8]:
import glob 
import pandas as pd
from datetime import datetime, timedelta
import os
import csv

In [2]:
src_path = '/well/doherty/users/cxx579/project_data/raine/gen1_26/sleepLabelsClean/'
time_dest_path = '/well/doherty/users/cxx579/project_data/raine/gen1_26/sleepTime.csv'

file_desp = src_path + '/*.csv'
file_list = glob.glob(file_desp)

In [30]:
# proper file names for gen1
# need to do these conversions

original_files = ['81112016_468044.csv', '28292016_504112.csv', '30352016_203796.csv',
                 '1412016-_514186.csv', '9032017-_162701.csv', '1412015-_667179.csv']
new_files = ['28112016_468044.csv', '28092016_504112.csv', '30052016_203796.csv',
            '1412016_514186.csv', '09032017_162701.csv', '14012015_667179.csv']
for i in range(len(original_files)):
    os.rename(src_path+original_files[i], src_path+new_files[i])

In [9]:
def parse_file_name(file_path):
    file_name = file_path.split('/')[-1]
    file_name = file_name[:-4] # remove .csv extension
    subjectID = file_name.split('_')[-1]
    file_date = file_name.split('_')[0]

    first_day = datetime.strptime(file_date, '%d%m%Y')

    if subjectID == '942099' or subjectID == '687006':
        # labels start on the same day:
        second_day = first_day
    else:
        second_day = first_day + timedelta(days=1)

    first_day_str = first_day.strftime('%d%m%Y')
    second_day_str = second_day.strftime('%d%m%Y')
    return first_day_str, second_day_str, subjectID

In [8]:
def removeLastNRows(csv_path, num_rows):
    f = open(csv_path, "r+")
    lines = f.readlines()
    for i in range(num_rows):
        lines.pop()

    f = open(csv_path, "w+")
    f.writelines(lines)
    f.close()
test_file = '/well/doherty/users/cxx579/project_data/raine/gen1_26/sleepLabelsClean/27042016_156814.csv'
removeLastNRows(test_file, 9)
# extra rows end of the file

In [10]:
subject_IDs = []
start_times = []
end_times = []

for my_file in file_list:
    try:
        first_day_str, second_day_str, subjectID = parse_file_name(my_file)

        if (subjectID == '585573'):
            labels = pd.read_csv(my_file,  names=['idx', 'time', 'slee_stage'],
                                index_col=False)
        else:
            labels = pd.read_csv(my_file)
            labels.reset_index(inplace=True)
            labels.columns = ['idx', 'time', 'slee_stage']

        start_t = labels.iloc[0]['time']
        end_t = labels.iloc[-1]['time']
        start_time = datetime.strptime(first_day_str+' '+start_t, '%d%m%Y %I:%M:%S %p') 
        end_time = datetime.strptime(second_day_str+' '+end_t, '%d%m%Y %I:%M:%S %p') 
        subject_IDs.append(subjectID)
        start_times.append(start_time)
        end_times.append(end_time)
    except Exception as e:
        print(e)
        print(my_file)
        print("File not parsed properly!")

Length mismatch: Expected axis has 2 elements, new values have 3 elements
/well/doherty/users/cxx579/project_data/raine/gen1_26/sleepLabelsClean/27042016_156814.csv
File not parsed properly!


In [12]:
sleepTimes = {'subject_id': subject_IDs, 'start_time': start_times, 'end_time': end_times}
sleepDf = pd.DataFrame(sleepTimes)
sleepDf.to_csv(time_dest_path)

# Start processing gen2
Still need to clean up the gen2 sleep labels
Like labels occur on the same days or csv malformed

In [58]:
src_path = '/well/doherty/users/cxx579/project_data/raine/gen2_22/sleepLabelsClean/'
time_dest_path = '/well/doherty/users/cxx579/project_data/raine/gen2_22/sleepTime.csv'

file_desp = src_path + '/*.csv'
file_list = glob.glob(file_desp)

In [76]:
subject_IDs = []
start_times = []
end_times = []

for my_file in file_list:
    try:
        first_day_str, second_day_str, subjectID = parse_file_name(my_file)

        labels = pd.read_csv(my_file)
        if subjectID != '01042014_223365':
            labels.reset_index(inplace=True)
        labels.columns = ['idx', 'time', 'slee_stage']

        start_t = labels.iloc[0]['time']
        end_t = labels.iloc[-1]['time']
        start_time = datetime.datetime.strptime(first_day_str+' '+start_t, '%d%m%Y %I:%M:%S %p') 
        end_time = datetime.datetime.strptime(second_day_str+' '+end_t, '%d%m%Y %I:%M:%S %p') 
        subject_IDs.append(subjectID)

        start_times.append(start_time)
        end_times.append(end_time)
    except:
        print(my_file)
        print("File not parsed properly!")

/well/doherty/users/cxx579/project_data/raine/gen2_22/sleepLabelsClean/01042014_223365.csv
File not parsed properly!


In [77]:
sleepTimes = {'subject_id': subject_IDs, 'start_time': start_times, 'end_time': end_times}
sleepDf = pd.DataFrame(sleepTimes)
sleepDf.to_csv(time_dest_path)